In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import os

In [2]:
os.chdir('C:\\Users\\alexx\\Documents\\thesis\\raw data')
df = pd.read_excel('facemasks_data_20200210.xlsx')
df=df.drop(['地域', '業態','対象店舗数','営業日数','登録日','最新売上日','大分類名', '大分類コード', 
            '小分類名', '小分類コード','１日当り金額', '１日当り個数', '１日当り容量', '出現店１日金額',
            '出現店１日個数','出現店１日容量','出現店販売日数','出現店千人金額', '出現店千人個数', 
            '出現店千人容量', '１店当り金額','１店当り個数', '１店当り容量', '出現店１店金額', '出現店１店個数', 
            '出現店１店容量','カバー率', '加重カバー率', '出現店来店客数', '販売店舗数','Unnamed: 51', 'Unnamed: 52'], axis = 1)

os.chdir('C:\\Users\\alexx\\Documents\\thesis\\processed data')

In [3]:
# Summary of raw sales data
df['date'] = df['年月日'].apply(lambda x: x.strftime('%Y-%m-%d'))

df["販売容量ALL"] = df["販売容量"] + df["特売販売容量"]
df_daily_sale = df.groupby('date').sum()['販売容量ALL'].reset_index()

df_daily_sale.mean()  

販売容量ALL    77387.597222
dtype: float64

# 1 Create Dependent Variable

In [4]:
# here we create "spike" as the dependent variable.
# 1 = days with sales higher than 150% of the average

def spike(row):
    if row['販売容量ALL'] >= (77387.597222*1.5):
        val = '1'
    else:
        val = '0'
    return val
df_daily_sale['spike'] = df_daily_sale.apply(spike, axis=1)

# 2 Create Independent Variables

## 2.1 Daiy Sales per Customer

In [5]:
# here we create "sale_pp" to represent daily saled per customer
df_daily_customer = df.groupby('date').mean()['来店客数'].reset_index()
df_daily_sale = pd.concat([df_daily_sale, df_daily_customer['来店客数']], axis=1)
df_daily_sale['sale_pp'] = df_daily_sale['販売容量ALL'] / df_daily_sale['来店客数']

df_daily_sale = df_daily_sale.set_index('date') 

## 2.2 Daily Sales by Materials

In [6]:
# First, we check every product name, and googled each of the product to check materials of facemasks.
df_name = pd.DataFrame(df['商　品　名'].unique())
df_name

,0
0,ＴＨＴ スーパーガードマスク 小 ６０枚
1,バンダイ アンパンマン ちいさな立体マスク ３枚
2,バンダイ アンパンマン ちいさな立体マスク のりもの編 ３枚
3,バンダイ 騎士竜戦隊リュウソウジャー こども不織布マスク ７枚
4,バンダイ アンパンマン ちいさな立体マスク おかおとおんぷ柄 ５枚
...,...
389,トレードワーク リーフフレッシュ ５段プリーツマスク くもりしらず レギュラー ５枚
390,クツワ 給食カラーマスク ＫＺ００６ＭＴ ミントグリーン ２枚
391,ショウワ ポケットモンスター マスク １枚
392,ショウワ ドラゴンボール超 マスク １枚


In [7]:
# Then, according to the names of products, we create a column to write down materials of each facemask.
def material(row):
    if 'シート' in row['商　品　名']:
        val = 'protective sheet'
    elif 'ガーゼ'in row['商　品　名'] or '給食'in row['商　品　名'] or'ショウワ'in row['商　品　名']:
        val = 'gauze'
    elif 'シルク'in row['商　品　名'] or 'アルファックス'in row['商　品　名']or'コットン'in row['商　品　名']or'立体布'in row['商　品　名']:
        val = 'cloth'
    elif '温感'in row['商　品　名']or'蒸気'in row['商　品　名']or'潤い'in row['商　品　名']or'のどぬーる'in row['商　品　名']or'美肌'in row['商　品　名']:
        val = 'heating/moisturizing'
    elif 'アラクス'in row['商　品　名']or'プロダクトＩ'in row['商　品　名']or'フロンティア'in row['商　品　名']:
        val = 'urethane'
    elif '不織布'in row['商　品　名']or'サージカル'in row['商　品　名']or'ＴＨＴ'in row['商　品　名']or'バンダイ'in row['商　品　名']or'ＤＲ．Ｃ'in row['商　品　名']or'Ｉオーヤマ'in row['商　品　名']or'Ｎ＆Ｎコーポ'in row['商　品　名']or'ＹＡＹＡ'in row['商　品　名']or'アズフィット'in row['商　品　名']or'イオンＴＶ'in row['商　品　名']or'エスパック'in row['商　品　名']or'サラヤ'in row['商　品　名']or'サンエイ'in row['商　品　名']or'システムポリマ'in row['商　品　名']or'スケーター'in row['商　品　名']or'スズラン'in row['商　品　名']or'チャーム'in row['商　品　名']or'トレードワン'in row['商　品　名']or'ネピア'in row['商　品　名']or'バイリーン'in row['商　品　名']or'パッショーネ'in row['商　品　名']or'ハピネット'in row['商　品　名']or'ビーエムシー'in row['商　品　名']or'ピエラス'in row['商　品　名']or'ピジョン'in row['商　品　名']or'ピップ'in row['商　品　名']or'メディカルＪ'in row['商　品　名']or'メディコム'in row['商　品　名']or'ヨコイ'in row['商　品　名']or'リブラボ'in row['商　品　名']or'八社会'in row['商　品　名']or'白十字'in row['商　品　名']or'白元'in row['商　品　名']or'大三'in row['商　品　名']or'東京企画'in row['商　品　名']or'横井定'in row['商　品　名']or'紀陽除虫菊'in row['商　品　名']or'近江兄弟'in row['商　品　名']or'柳瀬'in row['商　品　名']or'日本ＰＤＩ'in row['商　品　名']or'森川'in row['商　品　名']or'小久保'in row['商　品　名']or'玉川'in row['商　品　名']or'興和'in row['商　品　名']or'原田産業'in row['商　品　名']or'トレードワーク'in row['商　品　名']or'大王'in row['商　品　名']:
        val = 'nonwoven'
    else:
        val = 'NULL'
    return val
df['material'] = df.apply(material, axis=1)

In [8]:
# Here, we create 6 binary variables to represent 6 materials. 
df_m = df.groupby(['date', 'material'])['販売容量ALL'].sum()
df_m = df_m.unstack(level=0)
df_m = df_m.fillna(0)
df_m = df_m.T
df_m['total'] = df_m[list(df_m.columns)].sum(axis=1)
df_m.reset_index(level=0, inplace=True)

df_m = df_m.replace([np.inf, -np.inf], np.nan)
df_m = df_m.replace(np.nan, 0)

## 2.2 Daily Share of Sales by Materials

In [9]:
# Here, we calculate sale shares of each material 
df_m = df_m.set_index('date')
df_m['total'] = df_m[list(df_m.columns)].sum(axis=1)
for x in df_m:
    df_m[x +'_share'] = df_m[x] / df_m['total']
df_m.reset_index(level=0, inplace=True)
df_m = df_m.set_index('date')

df_m = df_m.drop(['total_share','total'], axis=1)
df_m.shape

(72, 10)

In [10]:
# Here, we check if there is any empty column.
df_m = df_m.loc[:, (df_m != 0).any(axis=0)]
print(df_m.shape)

(72, 10)


## 2.3 Daily Sales by Size

In [11]:
# Here, according to the names of products, we create a column to write down sizes of each facemask.
def size(row):
    if '温感' in row['商　品　名']or'小林薬'in row['商　品　名']or'蒸気'in row['商　品　名']or'潤い'in row['商　品　名']or'シート'in row['商　品　名']:
        val = 'other'
    elif '小顔'in row['商　品　名'] or '小久保'in row['商　品　名'] or'レギュラー'in row['商　品　名']or'ふつう'in row['商　品　名']:
        val = 'regular'
    elif '小'in row['商　品　名'] or 'スモール'in row['商　品　名']:
        val = 'smaller'
    elif 'アンパンマン'in row['商　品　名']or'こども'in row['商　品　名']or'キッズ'in row['商　品　名']or'バンダイ'in row['商　品　名']or'サンエイ'in row['商　品　名']or'ショウワ'in row['商　品　名']or'スケーター'in row['商　品　名']or'子供'in row['商　品　名']or'低学年'in row['商　品　名']or'園児'in row['商　品　名']or'ハピネット'in row['商　品　名']or'はじめて'in row['商　品　名']:
        val = 'kids'
    elif '大'in row['商　品　名']:
        val = 'bigger'
    else:
        val = 'regular'
    return val
df['size'] = df.apply(size, axis=1)

In [12]:
# Here, we create 4 binary variables to represent 4 sizes. 
df_s = df.groupby(['date', 'size'])['販売容量ALL'].sum()
df_s = df_s.unstack(level=0)
df_s = df_s.fillna(0)
df_s = df_s.T
df_s['total'] = df_s[list(df_s.columns)].sum(axis=1)
df_s.reset_index(level=0, inplace=True)

df_s = df_s.replace([np.inf, -np.inf], np.nan)
df_s = df_s.replace(np.nan, 0)

## 2.4 Daily Share of Sales by Sizes

In [13]:
# Here, we calculate sale shares of each size 
df_s = df_s.set_index('date')
for x in df_s:
    df_s[x +'_share'] = df_s[x] / df_s['total']
df_s.reset_index(level=0, inplace=True)

df_s = df_s.drop(['total_share','total'], axis=1)
df_s = df_s.set_index('date') 
df_s.shape

(72, 10)

In [14]:
# Here, we check if there is any empty column.
df_s = df_s.loc[:, (df_s != 0).any(axis=0)]
print(df_s.shape)

(72, 10)


## 2.5 Daily Sales by Design

In [15]:
# Here, according to the names of products, we create a column to write down designs of each facemask. "Yes" represents there is prints and/or colors on the facemasks; "no" represents plain facemasks.
def color(row):
    if '温感' in row['商　品　名']or'小林薬'in row['商　品　名']or'蒸気'in row['商　品　名']or'潤い'in row['商　品　名']or'シート'in row['商　品　名']:
        val = 'other'
    elif 'カラー'in row['商　品　名']or'色'in row['商　品　名']or'美フィット'in row['商　品　名']or'バンダイ'in row['商　品　名']or'サンエイ'in row['商　品　名']or'ショウワ'in row['商　品　名']or'スケーター'in row['商　品　名']or'クツワ'in row['商　品　名']or'薔薇'in row['商　品　名']or'ピンク'in row['商　品　名']or'グレー'in row['商　品　名']or'イエロー'in row['商　品　名']or'グリーン'in row['商　品　名']or'ラベンダー'in row['商　品　名']or'動物'in row['商　品　名']or'ハピネット'in row['商　品　名']or'香る'in row['商　品　名']or'アナ'in row['商　品　名']or'ラプンツェル'in row['商　品　名']or'ドナルド'in row['商　品　名']or'ベル'in row['商　品　名']or'ミッキー'in row['商　品　名']or'ミニ'in row['商　品　名']or'メイク'in row['商　品　名']:
        val = 'yes'
    else:
        val = 'no'
    return val
df['color'] = df.apply(color, axis=1)

In [16]:
# Here, we create 2 binary variables to represent 2 types of designs. 
df_c = df.groupby(['date', 'color'])['販売容量ALL'].sum()
df_c = df_c.unstack(level=0)
df_c = df_c.fillna(0)
df_c = df_c.T
df_c['total'] = df_c[list(df_c.columns)].sum(axis=1)
df_c.reset_index(level=0, inplace=True)

df_c = df_c.replace([np.inf, -np.inf], np.nan)
df_c = df_c.replace(np.nan, 0)

## 2.6 Daily Share of Sales by Design

In [17]:
# Here, we calculate sale shares of each design. 
df_c = df_c.set_index('date')
for x in df_c:
    df_c[x +'_share'] = df_c[x] / df_c['total']
df_c.reset_index(level=0, inplace=True)

df_c = df_c.drop(['other','other_share','total_share','total'], axis=1)
df_c = df_c.set_index('date') 
df_c.shape

(72, 4)

In [18]:
# Here, we check if there is any empty column.
df_c = df_c.loc[:, (df_c != 0).any(axis=0)]
print(df_c.shape)

(72, 4)


## 2.7 Daily Sales by Manufacturers

In [19]:
# Here, we calculate daily sales for each brand, or manufacturer.
df_b = pd.DataFrame(df.groupby(['date', 'メーカー名'])['販売容量ALL'].sum())
df_b = df_b.unstack(level=0)
df_b = df_b.T
df_b['total'] = df_b[list(df_b.columns)].sum(axis=1)
df_b.reset_index(level=0, inplace=True)

df_b = df_b.replace([np.inf, -np.inf], np.nan)
df_b = df_b.replace(np.nan, 0)

df_b=df_b.drop('level_0', axis=1)
df_b=df_b.drop('total', axis=1)

df_b = df_b.astype(bool).astype(int)

In [20]:
# Since some of the brands' name is too long, and R studio cannot display Japanese characters, we transfer brands' names in Japanese to numbers such as "Brand1" and "Brand2" to avoid later display problems.
for x in range(0,48):
    df_b.rename(columns={ df_b.columns[x]: "Brand"+str(x) }, inplace = True) 

df_b.shape

(72, 48)

In [21]:
# Here, we check if there is any empty column.
df_b = df_b.loc[:, (df_b != 0).any(axis=0)]
print(df_b.shape)

(72, 48)


##  2.8 Sold Packages by Package Size

In [22]:
# Here, we calculate sold packages by package sizes.
df_p = df[['date', '商　品　名', '平均価格','販売個数', '販売容量']].copy()
df_p = df_p.sort_values(by='date').reset_index(drop=True)

In [23]:
# Here, we check if their is any column with less than 5% sample size.
df_p_test = pd.DataFrame(df_p.groupby(['商　品　名','date'])['平均価格'].mean())
df_p_test = df_p_test.unstack(level=0)
for column in df_p_test:
    count = (df_p_test[column] == 0).sum()
    if count > 9:
        print('There is columns less than 5% of the sample size.')
# No output. 

In [24]:
df_p = df_p[(df_p != 0).all(1)]

col1 = pd.DataFrame(df_p.groupby(['date','商　品　名'])['平均価格'].mean())
col2 = pd.DataFrame(df_p.groupby(['date','商　品　名'])['販売個数'].sum())
col3 = pd.DataFrame(df_p.groupby(['date','商　品　名'])['販売容量'].sum())

df_p = pd.concat([col1, col2, col3], axis=1)

df_p['#in'] = df_p['販売容量']/df_p['販売個数']
df_p['price_per_unit'] = df_p['平均価格'] / df_p['#in']

df_p.reset_index(inplace=True)
df_p.sort_values(by=['date'], inplace=True, ascending=False)

In [25]:
# "Zero_three" represents the number of facemasks in each package is 0 < x <= 3 
# "three_ten" represents the number of facemasks in each package is 3 < x <= 10 
# "Ten_thirty" represents the number of facemasks in each package is 10 < x <= 30 
# "Over_thirty" represents the number of facemasks in each package is x > 30 

df_package_size = df_p
def package_size(row):
    if row['#in']<=3:
        val = 'zero_three'
    elif row['#in']<=10:
        val = 'three_ten'
    elif row['#in']<=30:
        val = 'ten_thirty'
    else:
        val = 'over_thirty'
    return val
df_package_size['package_size'] = df_package_size.apply(package_size, axis=1)

In [26]:
df_package_by_ps = pd.DataFrame(df_package_size.groupby(['date','package_size'])['販売個数'].mean())
df_package_by_ps = df_package_by_ps.unstack(level=0).T.fillna(0)

df_package_by_ps.reset_index(level=0, inplace=True)
df_package_by_ps = df_package_by_ps.drop('level_0', axis=1)

for x in df_package_by_ps:
    df_package_by_ps = df_package_by_ps.rename(columns={x: x+'_package_number'}) 
    
df_package = pd.concat([df_package_by_ps], axis=1)
df_package.shape

(72, 4)

In [27]:
# Here, we check if there is any empty column.
df_package = df_package.loc[:, (df_package != 0).any(axis=0)]
print(df_package.shape)

(72, 4)


## 2.9 Daily Average Price per Unit

In [28]:
df_p = pd.DataFrame(df_p.groupby('date').mean()['price_per_unit'])
df_p = pd.DataFrame(df_p['price_per_unit'])

# 3 Create Time-Based Features

In [29]:
# We combine all variables created before as a big data frame to prepare for later time-based features.
df_num = pd.concat([df_m, df_s, df_c, df_p, df_package], axis=1)
dfALL = pd.concat([df_num, df_b], axis=1)

## 3.1 Lags

In [30]:
#### lag1, Lag2, Lag3 ####
LAG1 = dfALL.shift(periods=1) 
LAG2 = dfALL.shift(periods=2) 
LAG3 = dfALL.shift(periods=3) 

#### lag difference ####
LAG_nu_1 = df_num.shift(periods=1) 
LAG_nu_2 = df_num.shift(periods=2) 
LAG_nu_3 = df_num.shift(periods=3) 

diff_LAG1 = LAG_nu_1 - df_num
diff_LAG2 = LAG_nu_2 - df_num
diff_LAG3 = LAG_nu_3 - df_num

for x in LAG1:
    LAG1 = LAG1.rename(columns={x: 'Lag1 '+x}) 
for x in LAG2:
    LAG2 = LAG2.rename(columns={x: 'Lag2 '+x})
for x in LAG3:
    LAG3 = LAG3.rename(columns={x: 'Lag3 '+x})
    
for x in diff_LAG1:
    diff_LAG1 = diff_LAG1.rename(columns={x: 'DiffLag1 '+x}) 
for x in diff_LAG2:
    diff_LAG2 = diff_LAG2.rename(columns={x: 'DiffLag2 '+x}) 
for x in diff_LAG3:
    diff_LAG3 = diff_LAG3.rename(columns={x: 'DiffLag3 '+x}) 

## 3.2 Percentage Change

In [31]:
df_pct = df_num.pct_change()

for x in df_pct:
    df_pct = df_pct.rename(columns={x: 'pct '+x}) 

df_pct = df_pct.replace([np.inf, -np.inf], np.nan)
df_pct = df_pct.fillna(0)

## 3.3 Rolling Window Features

In [32]:
##### rolling 3 ####
ROLL_3 = df_num.rolling(3, win_type='triang').mean()

#### rolling difference #### 
DiffRoll3 = ROLL_3 - df_num

for x in ROLL_3:
    ROLL_3 = ROLL_3.rename(columns={x: 'Roll3 '+x}) 
for x in DiffRoll3:
    DiffRoll3 = DiffRoll3.rename(columns={x: 'DiffRoll3 '+x}) 

# 4 Output All Variables 

In [33]:
dfALL = pd.concat([df_daily_sale['spike'], df_daily_sale['sale_pp'], dfALL,
                   df_pct, LAG1, LAG2, LAG3, diff_LAG1, diff_LAG2, diff_LAG3, ROLL_3, DiffRoll3], axis=1)
dfALL = dfALL.iloc[:72, :]

In [34]:
dfALL = dfALL.iloc[3:,:]
dfALL = dfALL.replace([np.inf, -np.inf], np.nan)
dfALL = dfALL.replace(np.nan, 0)
print(dfALL.shape)

dfALL = dfALL.loc[:, (dfALL != 0).any(axis=0)]
print(dfALL.shape)

(69, 484)
(69, 483)


In [35]:
os.chdir('C:\\Users\\alexx\\Documents\\thesis\\processed data')
dfALL.to_csv('SALES.csv')

In [36]:
# check every feature name in the data frame.
for i in dfALL.columns:
    print(i)

spike
sale_pp
cloth
gauze
heating/moisturizing
nonwoven
urethane
cloth_share
gauze_share
heating/moisturizing_share
nonwoven_share
urethane_share
bigger
kids
other
regular
smaller
bigger_share
kids_share
other_share
regular_share
smaller_share
no
yes
no_share
yes_share
price_per_unit
over_thirty_package_number
ten_thirty_package_number
three_ten_package_number
zero_three_package_number
Brand0
Brand1
Brand2
Brand3
Brand4
Brand5
Brand6
Brand7
Brand8
Brand9
Brand10
Brand11
Brand12
Brand13
Brand14
Brand15
Brand16
Brand17
Brand18
Brand19
Brand20
Brand21
Brand22
Brand23
Brand24
Brand25
Brand26
Brand27
Brand28
Brand29
Brand30
Brand31
Brand32
Brand33
Brand34
Brand35
Brand36
Brand37
Brand38
Brand39
Brand40
Brand41
Brand42
Brand43
Brand44
Brand45
Brand46
Brand47
pct cloth
pct gauze
pct heating/moisturizing
pct nonwoven
pct urethane
pct cloth_share
pct gauze_share
pct heating/moisturizing_share
pct nonwoven_share
pct urethane_share
pct bigger
pct kids
pct other
pct regular
pct smaller
pct bigger_

# 5 Check Linearity

In [37]:
# it takes long time, so we won't run it in the demonstration. 

#col_choice = dfALL.columns

#for x in col_choice:
    #plt.scatter(dfALL[x], dfALL['spike'])
    #plt.title(x)
    #plt.show()